## Apress - Industrialized Machine Learning Examples

Andreas Francois Vermeulen
2019

### This is an example add-on to a book and needs to be accepted as part of that copyright.

# Chapter-015-06-Mars-Mines-01

In [ ]:
import os
import datetime
import pandas as pd

### Part A - Setup Simulation

In [ ]:
LOADBAYCNT = 4           # Amount of Load Bays
#NEW_HOPPERS = 1000000    # Total number of Mars Hoppers
NEW_HOPPERS = 1000       # Total number of Mars Hoppers
INTERVAL_HOPPERS = 20.0  # Generate new Hopper roughly every x seconds
MIN_WAITLIMIT = 15       # Min. Hopper WAITLIMIT
MAX_WAITLIMIT = 20       # Max. Hopper WAITLIMIT
#DISPLAYRESULTS = True
DISPLAYRESULTS = False

In [ ]:
nowStart = datetime.datetime.now()

In [ ]:
dirname = '../../Results/Chapter 15/999-DL/100-Raw-Zone/'

# Create 100-Raw-Zone

![Raw Zone](../../images/DL-RZ.JPG)

In [ ]:
pathRealName=os.path.realpath(dirname)
print(pathRealName)

In [ ]:
if not os.path.exists(pathRealName):
    print('Make:', pathRealName)
    os.makedirs(pathRealName)

### Part B - Make Hopper

# Create Mars Hopper

In [ ]:
filename1='15-06-Retrieve-Mars-Hopper.csv'
fileZipname1=filename1 + '.gz'
print(fileZipname1)

In [ ]:
fullRealZipName1 = os.path.join(pathRealName, fileZipname1)

In [ ]:
t=0
hopper_list=[]

In [ ]:
hoppertypes = pd.DataFrame(['silicon', 
                            'hydrogen', 
                            'oxygen', 
                            'iron', 
                            'magnesium', 
                            'manganese', 
                            'aluminum', 
                            'calcium', 
                            'gold', 
                            'silver', 
                            'nickel', 
                            'titanium', 
                            'platinum',
                            'copper']
                          )
hoppertypes.columns = ['HopperType']

In [ ]:
print('Hopper Types: %d' % (hoppertypes.shape[0]))

In [ ]:
print(hoppertypes.head(20))

### Part C - Simulate Hopper Traffic

In [ ]:
import random
import simpy

In [ ]:
RANDOM_SEED = 1968

In [ ]:
def source(env, number, interval, counter):
    """Source generates HOPPERS randomly"""
    for i in range(number):
        n = i + 1
        c = Hopper(env, 'Hopper-%04d' % n, counter, time_in_hopper=50.0)
        env.process(c)
        t = random.expovariate(1.0 / interval)
        yield env.timeout(t)

In [ ]:
def Hopper(env, name, counter, time_in_hopper):
    """Hopper arrives, is served and leaves."""
    
    i=random.randint(0, 5)
    hoppertype = str(hoppertypes.iloc[i][0])
    
    tonnage=random.randint(20, 100)
    
    loadbayid = random.randint(1,LOADBAYCNT)
    
    arriveenv = env.now
    if DISPLAYRESULTS: print('%7.4f %s: Hopper with %s is here' % (arriveenv, name, hoppertype))
    hopperRec=[arriveenv, 'Hopper Arrive', name, hoppertype, tonnage, loadbayid, 0.0001]
    hopper_list.append(hopperRec)

    with counter.request() as req:
        WAITLIMIT = random.uniform(MIN_WAITLIMIT, MAX_WAITLIMIT)
        # Wait for the counter or abort at the end of our tether
        results = yield req | env.timeout(WAITLIMIT)

        wait = env.now - arriveenv

        if req in results:
            # We got to the counter
            waitenv=env.now            
            wait = waitenv - arriveenv
            if wait == 0: wait = 0.0001
            
            if DISPLAYRESULTS: print('%7.4f %s: Hopper with %s Waited %6.4f' % (waitenv, name, hoppertype, wait))
            
            hopperRec=[waitenv, 'Hopper Wait', name, hoppertype, tonnage, loadbayid, wait]
            hopper_list.append(hopperRec)

            tib = random.expovariate(1.0 / time_in_hopper)
            yield env.timeout(tib)
            
            doneenv=env.now
            wait = doneenv - arriveenv
            if wait == 0: wait = 0.0001
            
            if DISPLAYRESULTS: print('%7.4f %s: Hopper with %s Finished' % (doneenv, name, hoppertype))
                        
            hopperRec=[doneenv, 'Hopper Load', name, hoppertype, tonnage, loadbayid, wait]
            hopper_list.append(hopperRec)

        else:
            # We withdraw
            lostenv=env.now
            
            wait = env.now - arriveenv
            if wait == 0: wait = 0.0001
            
            if DISPLAYRESULTS: print('%7.4f %s: Hopper with %s Lost after %6.4f' % (lostenv, name, hoppertype, wait))
            hopperRec=[lostenv, 'Hopper Lost', name, hoppertype, tonnage, loadbayid, wait]
            hopper_list.append(hopperRec)

### Part - D - Setup and start the Hopper simulation

In [ ]:
print('Hopper Loading Started')
random.seed(RANDOM_SEED)
env = simpy.Environment()

# Start processes and run
loadbays = simpy.Resource(env, capacity=LOADBAYCNT)
env.process(source(env, NEW_HOPPERS, INTERVAL_HOPPERS, loadbays))
env.run()

print('Hopper Loading Done')

In [ ]:
hopperDF = pd.DataFrame(hopper_list)
hopperDF.columns = ['Time','Event','ObjectHopper','ObjectOre','Tonnage', 'LoadBayID','WaitTime']
hopperDF.index.name = 'ID'

In [ ]:
print(hopperDF.shape)

In [ ]:
if DISPLAYRESULTS: print(hopperDF.head(20))

In [ ]:
def marstime(s):
    #set to 24 hours, 39 minutes, and 35 seconds
    marsdayseconds = (24*60*60)+(39*60)+35
    marsday = round(s/marsdayseconds,6)
    return marsday

In [ ]:
simtime = max(hopperDF['Time'])
realworldtime = marstime(simtime)
print('Simulation of %0.6f seconds => yields %0.6f of real days of Mars Mining time' % (simtime, realworldtime))

# Save Mars Hopper

In [ ]:
print(fullRealZipName1)
hopperDF.to_csv(fullRealZipName1, index=False, encoding='utf-8', compression='gzip')

#  Create Mars Mines

In [ ]:
filename2='10-06-Retrieve-Mars-Mines.csv'
fileZipname2=filename2 + '.gz'
print(fileZipname2)

In [ ]:
fullRealZipName2 = os.path.join(pathRealName, fileZipname2)

### Part - E - Setup Mars Mines

In [ ]:
minesDF = pd.DataFrame(
[
    [0,0,0],
    [1,-88,-27],
    [2,77,77],
    [3,-19,68],
    [4,20,19]
]
)
minesDF.columns = ['MineID', 'Longitude', 'Latitude']
minesDF.index.name = 'ID'

In [ ]:
print(minesDF.shape)

In [ ]:
print(minesDF.head())

# Save Mars Mines

In [ ]:
print(fullRealZipName2)

In [ ]:
minesDF.to_csv(fullRealZipName2, index=False, encoding='utf-8', compression='gzip')

# Show Progress

In [ ]:
nowStop = datetime.datetime.now()
runTime=nowStop-nowStart
print('Start:', nowStart.strftime('%Y-%m-%d %H:%M:%S'))
print('Stop: ', nowStop.strftime('%Y-%m-%d %H:%M:%S'))
print('Time: ', runTime)